# Import Packages

In [1]:
import torch
from glob import glob
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, SubsetRandomSampler
from torch.utils.data.dataset import Dataset
from torchvision import transforms
import torch.optim as optim
from torch import nn
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np

# General helper functions

In [2]:
def freeze_all(model_params):
    for param in model_params:
        param.requires_grad = False
        
def unfreeze_all(model_params):
    for param in model_params:
        param.requires_grad = True
        
def get_trainable(model_params):
    return (p for p in model_params if p.requires_grad)

def load_image(filename) :
    img = Image.open(filename)
    img = img.convert('RGB')
    return img

# Load the Resnet Model

In [3]:
resnet = torch.hub.load('pytorch/vision', 'resnet34', pretrained=True)
resnet.eval()

Using cache found in C:\Users\kuvh/.cache\torch\hub\pytorch_vision_main
C:\Users\kuvh\.conda\envs\cuda\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\kuvh\.conda\envs\cuda\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet34-b627a593.pth" to C:\Users\kuvh/.cache\torch\hub\checkpoints\resnet34-b627a593.pth
28.1%IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change 